##### 实用技巧：

1. 布尔值也是**数字**：`True`等价于`1`，`False`等价于`0`。

```python
#可配合sum函数在需要计算总数时简化操作
l = [1,2,4,5,7]
total_ou = sum(i % 2 == 0 for i in l)

#将某个布尔值当做列表的下标使用，可实现类似三元表达式的目的
result = ["Python", "Javascript"][2 > 1]

```

2. **使用括号将长字符串包起来，可以随意拆行**。

```python
s = (
    "There is something really bad happened during the process. "
    "Please contact your administrator."
)

def main():
    logger.info(
        "There is something really bad happened during the process. "
        "Please contact your administrator."
    )
```
3. 使用`textwrap`库解决带缩进层级的代码里的长字符串问题。

```python
for textwrap import dedent

def main():
    if user.is_active:
        # dedent 将会缩进整段文字最左边的空字符串
        message = dedent("""\
            Welcome, today's movie list:
            - Jaw (1975)
            - The Shining (1980)
            - Saw (2004)""")
```
4. 特别大的数字，可以通过在中间添加下划线提高可读性。

```python
# 以“千”为单位划分数字
bill = 10_000_000.0
# 16进制数字同样有效，4个一组更易读
num1 = 0xCAFE_F00D
# 二进制也有效
num2 = 0b_0011_1111_0100_1110
# 处理字符串的时候也会正确处理下划线
num3 = int('0b_1111_0000', 2)
```

5. 使用“无穷大”float("inf")：`float("-inf") < 任意数值 < float("inf")`

```python
# A.根据年龄升序排序，没有提供年龄的放在最后边
users = {"tom": 19, "jenny": 13, "jack": None, "andrew": 43}
sorted_users = sorted(users.keys(), key=lambda user: users.get(user) or float('inf'))

# B. 作为循环初始值，简化第一次判断逻辑(特别是在不能明确实际数据的范围时)
max_num = float('-inf')
# 找到列表中最大的数字
for i in [23,71,3,21,8]:
    if i > max_num:
        max_num = i
```

##### 常见误区

1. `value += 1`并非线程安全：

“线程安全”：通常被用来形容**某个行为或者某类数据结构，可以在多线程环境下被共享使用并产生预期内的结果**。

一个典型的满足“线程安全”的模块是`queue`队列模块。

> 多线程环境下的编码非常复杂，要足够谨慎，不能相信自己的直觉。

### 容器之道

**容器**：*对专门用来装其他对象的数据类型的统称。*

常见的内建容器类型：
* 列表（list）
* 元组（tuple）
* 字典（dict）
* 集合（set）

##### 写更快的代码

1. 避免频繁扩充列表/创建新列表

> 在Python的实现细节里面，列表的内存是按需分配的，当某个列表当前拥有的内存不够时，便会触发内存扩容逻辑。而分配内存是一项昂贵的操作。

避免过于频繁的内存分配：
* 更多的使用`yield`关键字，返回生成器对象
* 尽量使用生成器表达式替代列表推导表达式
    * 生成器表达式：`(i for i in range(100))`
    * 列表推导表达式：`[i for i in range(100)]`
* 尽量使用模块提供的懒惰对象：
    * 使用`refinditer`替代`re.findall`
    * 直接使用可迭代的文件对象：`for line in fp`，而不是`for line in fp.readlines()`

2. 在列表头部操作多的场景使用`deque`模块

如需在列表头部操作多，使用`collections.deque`类型替代列表。

3. 使用集合/字典来判断成员是否存在

> 因为字典与集合都是基于哈希表（Hash Table）数据结构实现的。

In [1]:
test = (i for i in range(1000000000000000000000))